In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pandas as pd
import torch
import pickle
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from IPython import display
import argparse

from timeit import default_timer

from lib.utilities3 import *
from lib.datahelper import *
from lib.dataset_raw import *
from lib.fno import *

from lib.memorytracking import save_memory_usage
import datetime

torch.manual_seed(0)
np.random.seed(0)

In [20]:
MODES = 8
WIDTH = 128
BATCH_SIZE = 8

# if we should use cuda
GPU = True
# if we should use full version of dataset
DATASET_BIG = False
EPOCHS = 1000
# how often to save model/residual graphs
SAVE_INTERVAL = 10

# change this on every experiment
EXPERIMENT = f"encoded-batchsize{BATCH_SIZE}-{'gpu' if GPU else 'cpu'}-{'BIG' if DATASET_BIG else 'SMALL'}-{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

In [21]:
print ("=" * 40)
print ("RUN DETAILS")
print ("=" * 40)

print (f"MODES={MODES}, WIDTH={WIDTH}, BATCH SIZE={BATCH_SIZE}, GPU={GPU}, BIG DATASET={DATASET_BIG}, EPOCHS={EPOCHS}, SAVE INTERVAL={SAVE_INTERVAL}")
print()

print ("=" * 40)
print ("BEGIN RUN")
print ("=" * 40)

filename = '../Data/DatVel5000_Sou10_Rec10_Dim100x100_Gradient_Encoded.npz'
if not DATASET_BIG:
    filename = '../Data/DatVel30_Sou100_Rec100_Dim100x100_Downsampled_Encoded.npz'

RUN DETAILS
MODES=8, WIDTH=128, BATCH SIZE=8, GPU=True, BIG DATASET=False, EPOCHS=1000, SAVE INTERVAL=10

BEGIN RUN


In [22]:
print ("loading dataset")
with np.load(filename, allow_pickle=True) as fid:
    train_data = EikonalDatasetRaw(fid, 'train')
    test_data = EikonalDatasetRaw(fid, 'test')

loading dataset


In [23]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader_single = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

In [24]:
learning_rate = 0.0005

kernel_metadata_file='./lib/largefile-kernels-metadata.pkl'

with open(kernel_metadata_file, 'rb') as f:
    kernel_metadata = pickle.loads(f.read())

y_normalizer = UnitGaussianNormalizer(
    meanval=kernel_metadata['mean'],
    stdval=kernel_metadata['std'],
    maxval=kernel_metadata['max'],
    minval=kernel_metadata['min'],
)

In [ ]:
######## TRAINING

print ("begin training!!")
fno_model = FNO2d(MODES, MODES, WIDTH)
if GPU:
    fno_model = fno_model.cuda()

test_fno = []
train_fno = []
log_test_fno = []
log_train_fno = []

optimizer = torch.optim.AdamW(fno_model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

myloss = LpLoss(size_average=False)

if GPU:
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.empty_cache()
    before_memory = torch.cuda.memory_allocated()

    t1 = default_timer()
    y_normalizer.cuda()

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

prefix = EXPERIMENT

for ep in range(1, EPOCHS + 1):
    print (f"epoch {ep}")
    if ep < 5 or ep % SAVE_INTERVAL == 0:
        save_models(fno_model, ep, MODES, WIDTH, prefix=prefix)

    fno_model.eval()
    test_l2 = 0.0

    with torch.no_grad():
        for x, y in test_loader:
            if GPU:
                x, y = x.cuda(), y.cuda()
            x, y = x.float(), y.float()

            out = fno_model(x)
            out = out.squeeze(-1)
            y = y_normalizer.decode(y)
            if GPU:
                y = y.cuda()

            out = y_normalizer.decode(out)

            curr_loss = myloss(out, y.view(len(y),-1)).item()
            test_l2 += curr_loss

    fno_model.train()
    train_l2 = 0
    for x, y in train_loader:
        if GPU:
            x, y = x.cuda(), y.cuda()
        x, y = x.float(), y.float()

        optimizer.zero_grad()
        out = fno_model(x)
        if GPU:
            out = out.cuda()
        out = out.squeeze(-1)
        out = y_normalizer.decode(out)
        if GPU:
            out = out.cuda()
        y = y_normalizer.decode(y)
        if GPU:
            y = y.cuda()

        loss = myloss(out, y.view(len(y),-1))
        loss.backward()

        optimizer.step()
        scheduler.step()
        train_l2 += loss.item()

    train_l2/= len(train_data)
    test_l2 /= len(test_data)
    print(f"epoch: {ep}, \tl2 train: {train_l2} \tl2 test: {test_l2}")

    train_fno.append(train_l2)
    test_fno.append(test_l2)
    log_train_fno.append(math.log(train_l2))
    log_test_fno.append(math.log(test_l2))

    save_losses(train_fno, test_fno, log_train_fno, log_test_fno, MODES, WIDTH, prefix=prefix)

    if ep < 5 or ep % SAVE_INTERVAL == 0:
        plot_loss_curves(train_fno, test_fno, log_train_fno, log_test_fno, MODES, WIDTH, ep, prefix=prefix)

t2 = default_timer()
max_memory = torch.cuda.max_memory_allocated() - before_memory
print(f'Max Memory Allocated: {max_memory} Time: {t2-t1}')

begin training!!
epoch 1
epoch: 1, 	l2 train: 0.9328428958892823 	l2 test: 0.981662070274353
epoch: 0, 	l2 train: 0.9328428958892823 	l2 test: 0.981662070274353
epoch 2
epoch: 2, 	l2 train: 0.8987031234741211 	l2 test: 0.930295093536377
epoch: 0, 	l2 train: 0.9328428958892823 	l2 test: 0.981662070274353
epoch: 1, 	l2 train: 0.8987031234741211 	l2 test: 0.930295093536377
epoch 3
epoch: 3, 	l2 train: 0.875385294342041 	l2 test: 0.8965552968978882
epoch: 0, 	l2 train: 0.9328428958892823 	l2 test: 0.981662070274353
epoch: 1, 	l2 train: 0.8987031234741211 	l2 test: 0.930295093536377
epoch: 2, 	l2 train: 0.875385294342041 	l2 test: 0.8965552968978882
epoch 4


### Garbage Collection

We need to clean up unused variables after every run because otherwise CUDA runs out of memory!

In [27]:
import gc
gc.collect()
torch.cuda.empty_cache()